# Машинный перевод и озвучивание видеозаписей на русском языке

## О проекте

[Есть](https://www.youtube.com/watch?v=p3lsYlod5OU&ab_channel=LexFridman) интересная беседа [Лекса Фридмана](https://en.wikipedia.org/wiki/Lex_Fridman) с [Михаилом Левиным](https://en.wikipedia.org/wiki/Michael_Levin_(biologist)). Несмотря, на русское происхождение обоих собеседников разговор они ведут на английском. Цель проекта перевести и озвучить беседу на русском языке.

В проекте используется [готовая расшифровка с таймингом](https://karpathy.ai/lexicap/0325-large.html), созданная с помощью пакета [OpenAI Wisper](https://github.com/openai/whisper).
Далее описывается парсинг расшифровки, перевод на русский язык, машинное озвучивание, выравнивание длины русских фрагментов с соответствующими оригинальными.

Данный подход можно использовать для перевода и озвучивания любой видеозаписи.

## Устанавливаем библиотеки

In [190]:
# Библиотека для работы c Google Translate
%pip install googletrans==3.1.0a0

# Библиотека для генерации tts
%pip install pyttsx3

# Прогресс-бар
%pip install tqdm


[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Импортируем необходимые для работы модули

In [191]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from scipy.io.wavfile import read, write
from googletrans import Translator

import pyttsx3
import requests
import numpy as np
import pandas as pd


## Парсинг расшифровки с таймингом

Для парсинга используем библиотеку `BeatifulSoup`:

In [192]:
url = 'https://karpathy.ai/lexicap/0325-large.html'
data = requests.get(url).text
soup = BeautifulSoup(data)

## Обработка английского текста

Выделим фрагменты текста и их тайминги в отдельные списки:

In [193]:
t_divs = soup.find_all('div', {'class': 't'})
en_text = []
for div in t_divs:
    en_text.append(div.text)
    # if len(div.text) < 66:
    #     text += '\n'

s_divs = soup.find_all('div', {'class': 's'})
timing = []
for div in s_divs:
    timing.append(div.a.text)

Для удовлетворительного качества необходимо переводить текст как минимум целыми предложениями. В качестве знаков для разделения используем точку и вопросительный знак.

In [194]:
def sign_finder(sign, str):
    '''Finds the last position of the sing in the string'''
    rev_pos = str[::-1].find(sign)
    if rev_pos == -1:
        return 0    
    pos = len(str) - rev_pos
    return pos
def pos_finder(text):
    signs = ['.', '?']
    pos = []
    for sign in signs:
        pos.append(sign_finder(sign, text))
    return max(pos)

Перенесём незаконченные фрагменты предложений к фрагментам, содержащим знаки конца предложения, и внесём соответствующие поправки в тайминг.

In [195]:
dense_text = []
new_timing = []
buffer = []
time = []
for i, text in enumerate(en_text):
    pos = pos_finder(text)
    if (pos == 0) & (i < len(en_text) - 1):
        buffer.append(text)
        time.append(timing[i])
    elif len(buffer) > 0:
        for i in range(len(buffer)):
            text = buffer[-1] + text
        dense_text.append(text)
        new_timing.append(time[0])
        buffer.clear()
        time.clear()
    else:
        dense_text.append(text)
        new_timing.append(timing[i])


Из данных списков создадим таблицу `Pandas`: 

In [196]:
dict = {'timing': new_timing, 'en_text': dense_text}
df = pd.DataFrame(dict)
df

,timing,en_text
0,00:00:00.000,turns out that if you train a planarian and t...
1,00:00:09.760,pretty much every deep question of life. For ...
2,00:00:14.800,"have true symmetry, they have a true brain, t..."
3,00:00:20.640,"these little, they're about, you know, maybe ..."
4,00:00:24.560,And they have a head and a tail. And the firs...
...,...,...
1456,02:59:18.200,Thank you for listening to this conversation ...
1457,02:59:22.200,please check out our sponsors in the descript...
1458,02:59:26.760,Charles Darwin in The Origin of Species. From...
1459,02:59:35.760,the most exalted object which we're capable o...


Тайминги переведем в формат `DateFrame` и вычислим продожительность оригинальных фрагментов, сохранив их в отдельный столбец:

In [197]:
format = '%H:%M:%S.%f'
def duration_calculator(row):
    '''Calculates duration speech fragment in a row'''
    idx = row.name
    time1 = row['timing'] + '000'
    if idx == len(df) - 1:
        return datetime.strptime("03:00:20.000000", format) - datetime.strptime(time1, format)    
    time2 = df.loc[idx + 1, 'timing'] + '000'
    return datetime.strptime(time2, format) - datetime.strptime(time1, format)

df['duration'] = df.apply(duration_calculator, axis=1)
df['duration'] = df['duration'].apply(lambda x: x.total_seconds())
df

,timing,en_text,duration
0,00:00:00.000,turns out that if you train a planarian and t...,9.76
1,00:00:09.760,pretty much every deep question of life. For ...,5.04
2,00:00:14.800,"have true symmetry, they have a true brain, t...",5.84
3,00:00:20.640,"these little, they're about, you know, maybe ...",3.92
4,00:00:24.560,And they have a head and a tail. And the firs...,6.08
...,...,...,...
1456,02:59:18.200,Thank you for listening to this conversation ...,4.00
1457,02:59:22.200,please check out our sponsors in the descript...,4.56
1458,02:59:26.760,Charles Darwin in The Origin of Species. From...,9.00
1459,02:59:35.760,the most exalted object which we're capable o...,11.84


Посмотрим на длину фрагментов:

In [198]:
df.en_text.apply(len).describe()

count    1461.000000
mean      126.861739
std        68.620897
min         5.000000
25%        91.000000
50%        96.000000
75%       178.000000
max       574.000000
Name: en_text, dtype: float64

Самый короткий 5 символов, а длинный -- 574. 

Для перевода фрагментов текста в отдельные предложения (группы предложений) и определения соответствующей им временной продолжительности мне будет удобнее работать со списками, а не таблицей.

In [199]:
text = list(df.en_text)
duration = list(df.duration)

Будем ориентироваться на знаки окончания предложения в конце фрагмента:

In [200]:
def min_pos_finder(text):
    '''Returns the first position of the signs in the text'''
    signs = ['.', '?']
    pos = []
    for sign in signs:
        pos.append(text.find(sign))
    if min(pos) == -1:
        return max(pos) + 1
    else:
        return min(pos) + 1

Собственно переносим, что нужно в тексте и вносим, соответствующие изменения в их длительность:

In [201]:
for i, row in enumerate (text):
    if i != 0:
        pos = min_pos_finder(row)
        head = row[:pos]
        tail = row[pos:]
        duration[i - 1] = duration[i - 1] + duration[i] * len(head) / len(row)
        duration[i] = duration[i] * len(tail) / len(row)
        text[i - 1] = text[i - 1] + head
        text[i] = tail         


In [202]:
size = 0
for row in text:
    size += len(row)

print('Количество знаков в исходном тексте: ', df.en_text.apply(len).sum())
print('Количество знаков итоговом тексте: ', size)
print('Исходная общая продолжительность: ', sum(duration))
print('Общая продолжительность после обработки: ', df.duration.sum())


Количество знаков в исходном тексте:  185345
Количество знаков итоговом тексте:  185345
Исходная общая продолжительность:  10820.000000000025
Общая продолжительность после обработки:  10820.0


Всё на месте. Возваращаем данные обратно в таблицу.

In [203]:
df['en_text'] = text
df['duration'] = duration
df

,timing,en_text,duration
0,00:00:00.000,turns out that if you train a planarian and t...,11.868571
1,00:00:09.760,"For one thing, they're similar to our ancesto...",6.991322
2,00:00:14.800,"They have lots of different internal organs, ...",5.700107
3,00:00:20.640,And they have a head and a tail.,2.134468
4,00:00:24.560,And the first thing is planaria are immortal....,4.133287
...,...,...,...
1456,02:59:18.200,"To support this podcast, please check out our...",3.486111
1457,02:59:22.200,And now let me leave you with some words from...,6.378182
1458,02:59:26.760,"From the war of nature, from famine and death...",12.921874
1459,02:59:35.760,"There's grandeur in this view of life, with i...",30.850525


## Перевод текста на русский

Переводим текст на русский:

In [204]:
translator = Translator()
ru_text = []
for row in tqdm(text):
    ru_text.append(translator.translate(row, src='en', dest='ru').text)

100%|██████████| 1461/1461 [04:55<00:00,  4.95it/s]


In [208]:
df

,timing,en_text,duration
0,00:00:00.000,turns out that if you train a planarian and t...,11.868571
1,00:00:09.760,"For one thing, they're similar to our ancesto...",6.991322
2,00:00:14.800,"They have lots of different internal organs, ...",5.700107
3,00:00:20.640,And they have a head and a tail.,2.134468
4,00:00:24.560,And the first thing is planaria are immortal....,4.133287
...,...,...,...
1456,02:59:18.200,"To support this podcast, please check out our...",3.486111
1457,02:59:22.200,And now let me leave you with some words from...,6.378182
1458,02:59:26.760,"From the war of nature, from famine and death...",12.921874
1459,02:59:35.760,"There's grandeur in this view of life, with i...",30.850525


Короткими фрагментами переводить данный текст всё же долго. Кроме того, вероятно переводчик может учитывать контекст, что положительным образом должно сказываться на переводе. Однако наш текстовый блок слишком большой для одномоментного перевода, переводчик не справиться. Желательно подобрать максимальный размер группы текстов опытным путём, у меня это 90. Меньше - дольше, а больше - переводчик не справляется. Однако необходимо вставить подходящий маркер для обратного разъединения строк с чем мне не удалось справиться. Но код, на всякий случай, оставляю.

In [205]:
# print(len(df))
# translator = Translator()
# ru_text = []

# batch_size = 80
# start = 0
# for i in tqdm(range(round(len(df) / batch_size))):    
#     batch = df.en_text[start:start+batch_size]
    
#     en_batch = '*'.join(batch)
#     ru_batch = translator.translate(en_batch, src='en', dest='ru').text
#     # print(1, ru_batch[:100])
#     ru_text.extend(ru_batch.split('*'))
#     start += batch_size
# len(ru_text)

In [206]:
for i, row in enumerate(ru_text):
    if row == '':
        ru_text.pop(i)
len(ru_text)

1461

Посмотрим что получилось:

In [55]:
for row in ru_text[:10]:
    print(row)

Оказывается, если вы дрессируете планарий, а затем отрубаете им головы, хвост регенерирует совершенно новый мозг, который все еще помнит исходную информацию. Я думаю, что планарии содержат ответы практически на все важные вопросы жизни.
Во-первых, они похожи на наших предков. Так что у них настоящая симметрия, у них настоящий мозг, они не похожи на дождевых червей, они, знаете ли, гораздо более продвинутая форма жизни.
У них много различных внутренних органов, но они такие маленькие, они размером примерно два сантиметра в сантиметре или два в размере.
И у них есть голова и хвост.
И во-первых, планарии бессмертны. Так они не стареют.
Нет такой вещи, как старая планария. Итак, это прямо говорит вам, что эти теории термодинамических ограничений продолжительности жизни неверны.
Это не так хорошо, со временем все деградирует.
Нет, планарии могут продолжаться, наверное, вы знаете, как долго они существуют около 400 миллионов лет, верно?
Итак, это настоящие, так что планарии в нашей лаборатор

Боже мой! "размером примерно два сантиметра в сантиметре или два в размере".

С некоторыми фрагментами текста неплохо справляется переводчик `DeepL`, но, к сожалению, перевод текстов более 5 000  знаков требует подписки, а для россиян, в данный момент, она не предусмотрена.

Изменения в текст перевода можно внести, например, так:

In [57]:
ru_text[0] = 'Оказывается, если обучить планарий, а затем отрезать им голову, хвост регенерирует совершенно новый мозг, который все еще помнит первоначальную информацию. Я думаю, что в планариях кроется ответ практически на все глубокие вопросы жизни.'
ru_text[2] = 'У них много различных внутренних органов, но они такие маленькие, они размером примерно два сантиметра на один.'

Создадим заново датафрейм с текстами на двух языках, их продолжительностью, и сохраним его, при необходимости, в `csv`-файл.

In [59]:
df = pd.DataFrame({'duration': duration, 'en_text': text, 'ru_text': ru_text})
df.to_csv('levin.csv', index=False)

## Машинное озвучивание текста и подготовка к монтажу

Можно загрузить таблицу из файла:

In [21]:
df = pd.read_csv('levin.csv')

,duration,en_text,ru_text
0,11.868571,turns out that if you train a planarian and t...,"Оказывается, если обучить планарий, а затем от..."
1,6.991322,"For one thing, they're similar to our ancesto...","Во-первых, они похожи на наших предков. Так чт..."
2,5.700107,"They have lots of different internal organs, ...","У них много различных внутренних органов, но о..."
3,2.134468,And they have a head and a tail.,И у них есть голова и хвост.
4,4.133287,And the first thing is planaria are immortal....,"И во-первых, планарии бессмертны. Так они не с..."
...,...,...,...
1456,3.486111,"To support this podcast, please check out our...","Чтобы поддержать этот подкаст, пожалуйста, озн..."
1457,6.378182,And now let me leave you with some words from...,А теперь позвольте мне оставить вас нескольким...
1458,12.921874,"From the war of nature, from famine and death...","Из войны природы, из голода и смерти непосредс..."
1459,30.850525,"There's grandeur in this view of life, with i...","Есть величие в этом взгляде на жизнь, с ее нес..."


In [22]:
for i in range(10):
    print(i, df.ru_text[i])

0 Оказывается, если обучить планарий, а затем отрезать им голову, хвост регенерирует совершенно новый мозг, который все еще помнит первоначальную информацию. Я думаю, что в планариях кроется ответ практически на все глубокие вопросы жизни.
1 Во-первых, они похожи на наших предков. Так что у них настоящая симметрия, у них настоящий мозг, они не похожи на дождевых червей, они, знаете ли, гораздо более продвинутая форма жизни.
2 У них много различных внутренних органов, но они такие маленькие, они примерно, два сантиметра на один размером.
3 И у них есть голова и хвост.
4 И во-первых, планарии бессмертны. Так они не стареют.
5 Не существует такого явления, как старая планария. Это говорит о том, что все эти теории термодинамических ограничений продолжительности жизни неверны.
6 Это не так хорошо, со временем все деградирует.
7 Нет, планарии могут продолжаться, наверное, вы знаете, как долго они существуют около 400 миллионов лет, верно?
8 Итак, это настоящие, так что планарии в нашей лабо

Я использую голос, который загрузил с сайта https://rhvoice.su/

In [23]:
tts = pyttsx3.init()

voices = tts.getProperty('voices')
  
for voice in voices:
    # to get the info. about various voices in our PC 
    print("Voice:")
    print("ID: %s" %voice.id)
    print("Name: %s" %voice.name)
    print("Age: %s" %voice.age)
    print("Gender: %s" %voice.gender)
    print("Languages Known: %s" %voice.languages)

Voice:
ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_RU-RU_IRINA_11.0
Name: Microsoft Irina Desktop - Russian
Age: None
Gender: None
Languages Known: []
Voice:
ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
Name: Microsoft Zira Desktop - English (United States)
Age: None
Gender: None
Languages Known: []
Voice:
ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\TokenEnums\RHVoice\Aleksandr-hq
Name: Aleksandr-hq
Age: None
Gender: None
Languages Known: []


In [24]:
# Задать голос по умолчанию

tts.setProperty('voice', 'ru') 
tts.setProperty('rate', 300)
tts.setProperty('volume', .8)

# Попробовать установить предпочтительный голос
RU_VOICE_ID = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\TokenEnums\RHVoice\Aleksandr-hq"
# RU_VOICE_ID = 'HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_RU-RU_IRINA_11.0'


tts.setProperty('voice', RU_VOICE_ID)

Проверим настройки:

In [129]:
speech = 'Привет! Давайте проверим скорость и качество озвучивания'
tts.say(speech)
tts.runAndWait()

Создадим набор звуковых файлов из текстовых фрагментов:

In [25]:
for i in range(len(df)):
    speech = df.ru_text[i]
    file_name = 'files/' + f'{i}.wav'
    tts.save_to_file(speech, file_name)
    tts.runAndWait()


## Обработка звука

Сравним длину полученных фрагментов с длительностью оригинальных фрагментов:

In [26]:
durs = []
for i in range(len(df)):
    file_name = 'files/' + f'{i}.wav'
    fs, data = read(file_name)
    durs.append(round(len(data) / fs, 3))   


In [27]:
sum(durs)

8105.3939999999975

In [28]:
df['ru_duration'] = durs

In [29]:
df['delta_duration'] = df['duration'] - df['ru_duration']

In [30]:
df

,duration,en_text,ru_text,ru_duration,delta_duration
0,11.868571,turns out that if you train a planarian and t...,"Оказывается, если обучить планарий, а затем от...",10.129,1.739571
1,6.991322,"For one thing, they're similar to our ancesto...","Во-первых, они похожи на наших предков. Так чт...",7.991,-0.999678
2,5.700107,"They have lots of different internal organs, ...","У них много различных внутренних органов, но о...",4.537,1.163107
3,2.134468,And they have a head and a tail.,И у них есть голова и хвост.,1.107,1.027468
4,4.133287,And the first thing is planaria are immortal....,"И во-первых, планарии бессмертны. Так они не с...",2.179,1.954287
...,...,...,...,...,...
1456,3.486111,"To support this podcast, please check out our...","Чтобы поддержать этот подкаст, пожалуйста, озн...",3.651,-0.164889
1457,6.378182,And now let me leave you with some words from...,А теперь позвольте мне оставить вас нескольким...,3.938,2.440182
1458,12.921874,"From the war of nature, from famine and death...","Из войны природы, из голода и смерти непосредс...",6.402,6.519874
1459,30.850525,"There's grandeur in this view of life, with i...","Есть величие в этом взгляде на жизнь, с ее нес...",15.357,15.493525


In [31]:
df.delta_duration.describe()

count    1461.000000
mean        1.858047
std         2.189053
min        -3.084271
25%         0.381708
50%         1.352974
75%         2.837824
max        19.159033
Name: delta_duration, dtype: float64

In [32]:
df.delta_duration.sum()

2714.6059999999998

In [33]:
df.delta_duration.sum() + df.ru_duration.sum()

10820.0

In [34]:
deltas = list(df.delta_duration)

In [35]:
time = 0
for i, delta in enumerate(deltas):
    
    if delta < 0:        
        time += delta
        deltas[i] = 0
    elif time < 0:
        if abs(time) <= delta:
            deltas[i] += time
            time = 0
        else:
            time += delta 
            deltas[i] = 0       

In [36]:
sum(deltas)

2714.606000000003

In [37]:
df['delta_duration'] = deltas

In [38]:
df

,duration,en_text,ru_text,ru_duration,delta_duration
0,11.868571,turns out that if you train a planarian and t...,"Оказывается, если обучить планарий, а затем от...",10.129,1.739571
1,6.991322,"For one thing, they're similar to our ancesto...","Во-первых, они похожи на наших предков. Так чт...",7.991,0.000000
2,5.700107,"They have lots of different internal organs, ...","У них много различных внутренних органов, но о...",4.537,0.163429
3,2.134468,And they have a head and a tail.,И у них есть голова и хвост.,1.107,1.027468
4,4.133287,And the first thing is planaria are immortal....,"И во-первых, планарии бессмертны. Так они не с...",2.179,1.954287
...,...,...,...,...,...
1456,3.486111,"To support this podcast, please check out our...","Чтобы поддержать этот подкаст, пожалуйста, озн...",3.651,0.000000
1457,6.378182,And now let me leave you with some words from...,А теперь позвольте мне оставить вас нескольким...,3.938,2.275293
1458,12.921874,"From the war of nature, from famine and death...","Из войны природы, из голода и смерти непосредс...",6.402,6.519874
1459,30.850525,"There's grandeur in this view of life, with i...","Есть величие в этом взгляде на жизнь, с ее нес...",15.357,15.493525


In [39]:
print(df.duration.sum())
print(df.ru_duration.sum() + df.delta_duration.sum())

10820.0
10820.0


Добавим после каждого вновь озвученного звукового фрагмента, продолжительностью меньше оригинального, заполненной тишиной спейсер, необходимой продолжительности:

In [42]:
fs = 22050
for i, delta in enumerate(deltas):
    if delta > 0:
        file_name = f'files/{i}-1.wav'
        multiplicator = int(delta * fs)
        spacer = np.array([0] * multiplicator)
        write(file_name, fs, spacer)

In [41]:
with open("list.txt", "w") as f:
    for i in range(len(df)):
        filename = f'files/{i}.wav'
        line = f"file '{filename}'\n"
        if deltas[i] > 0:
            line += f"file 'spaces/{i}.wav'\n"
        f.write(line)

In [118]:
os.system("ffmpeg -f concat -i out/list.txt -c copy out/output.wav")

1

In [111]:

spacer = np.array([0] * 10 * 22050)
write('spaces/test.wav', fs, spacer)

In [114]:

req = 'ffmpeg -i files/test.wav concat -i spaces/test.wav out/test.wav'
os.system(req)


1

In [ ]:
grouped_df.apply(lambda row: add_spacer(row), axis=1)

In [ ]:
grouped_df

In [ ]:
total, fs = sf.read('files/0.wav')

for i in range(30):
    file_name = 'files/' + f'{i}.wav'
    print(file_name)
    data, fs = sf.read(file_name)
    total = np.append(total, data)
sf.write('files/total_1.ogg', total, fs)

In [ ]:
fs, data = read('files/total_1.wav')
Audio(data, rate=fs)

In [ ]:
write('files/total_10.wav', fs, data)

In [ ]:
fs, data_0 = read('files/0.wav')
fs, data_1 = read('files/1.wav')


In [ ]:
total = np.append(data_0, data_1)

In [ ]:
Audio(total, rate=fs)

In [ ]:
write('files/total.wav', fs, total)

In [ ]:
len(data) / fs / 3600

In [ ]:
import soundfile as sf
x, Fs = sf.read('files/0.wav')
print(len(x))